# K-Means Clustering – Stock & Market-Day Segmentation

Unsupervised analysis with two complementary views:

### Part 1 – Stock Behavioral Clustering
Aggregate each ticker's trading history into a summary profile (mean return, volatility, Sharpe proxy, volume behaviour, momentum, sentiment) and cluster tickers into behavioral archetypes.

### Part 2 – Market Day Clustering
Aggregate across tickers each day (cross-sectional mean return, dispersion, breadth, VIX, yield spread) and cluster days into market environment types.

Cluster labels can later be used as additional features in the supervised models.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 5)
RANDOM_STATE = 42

## 1. Load Data & Derive Base Features

In [ ]:
df = pd.read_csv('../data/merged_dataset.csv', parse_dates=['date'])
df = df.sort_values(['ticker', 'date']).reset_index(drop=True)
print(f'Shape: {df.shape}')

# ── Derived columns used by both parts ────────────────────────────────────────
df['price_to_ma20'] = (
    df['close'] / df['rolling_mean_20'].replace(0, np.nan)
).replace([np.inf, -np.inf], np.nan) - 1

df['hl_range'] = (df['high'] - df['low']) / df['close'].replace(0, np.nan)

df['vol_20ma'] = df.groupby('ticker')['volume'].transform(
    lambda x: x.rolling(20, min_periods=1).mean()
)
df['vol_norm'] = (df['volume'] / df['vol_20ma'].replace(0, np.nan)).clip(0, 10)

# Impute sparse columns with neutral values
df['VIX']            = df['VIX'].fillna(df['VIX'].median())
df['Yield_Spread']   = df['Yield_Spread'].fillna(df['Yield_Spread'].median())
df['sentiment_mean'] = df['sentiment_mean'].fillna(0)

# Binary: was next-day return positive?
df['up'] = (df.groupby('ticker')['daily_return'].shift(-1) > 0).astype(float)

print('Base features derived.')

---
## Part 1 – Stock Behavioral Clustering

Summarise each ticker over its full history into a fixed-length feature vector.

In [ ]:
ticker_stats = df.groupby('ticker').agg(
    mean_return       = ('daily_return',   'mean'),
    std_return        = ('daily_return',   'std'),
    mean_hl_range     = ('hl_range',       'mean'),
    mean_vol_norm     = ('vol_norm',       'mean'),
    mean_price_to_ma  = ('price_to_ma20',  'mean'),
    mean_sentiment    = ('sentiment_mean', 'mean'),
    up_rate           = ('up',             'mean'),   # win rate
    log_marketcap     = ('Marketcap',      lambda x: np.log1p(x.dropna().median())),
    revenuegrowth     = ('Revenuegrowth',  'median'),
    weight            = ('Weight',         'first'),
).dropna()

# Sharpe-like ratio
ticker_stats['sharpe_approx'] = (
    ticker_stats['mean_return'] / (ticker_stats['std_return'] + 1e-9)
)

# Also keep Sector for later interpretation
sector_map = df.groupby('ticker')['Sector'].first()
ticker_stats = ticker_stats.join(sector_map)

print(f'Tickers in clustering set: {len(ticker_stats)}')
ticker_stats.head()

### Elbow Method & Silhouette Scores

In [ ]:
TICKER_FEAT_COLS = [
    'mean_return', 'std_return', 'sharpe_approx',
    'mean_hl_range', 'mean_vol_norm', 'mean_price_to_ma',
    'mean_sentiment', 'up_rate',
    'log_marketcap', 'revenuegrowth', 'weight',
]

X_tickers = ticker_stats[TICKER_FEAT_COLS].dropna()
print(f'Tickers after dropna: {len(X_tickers)}')

scaler_t = StandardScaler()
X_tickers_s = scaler_t.fit_transform(X_tickers)

inertias, sil_scores = [], []
K_RANGE = range(2, 13)

for k in K_RANGE:
    km = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
    labels = km.fit_predict(X_tickers_s)
    inertias.append(km.inertia_)
    if k <= 8:
        sil_scores.append(silhouette_score(X_tickers_s, labels, sample_size=min(len(X_tickers_s), 2000)))
    else:
        sil_scores.append(None)

fig, axes = plt.subplots(1, 2, figsize=(14, 4))
axes[0].plot(list(K_RANGE), inertias, marker='o')
axes[0].set_title('Elbow Method – Stock Clustering')
axes[0].set_xlabel('k')
axes[0].set_ylabel('Inertia')

sil_vals = [s for s in sil_scores if s is not None]
sil_ks   = [k for k, s in zip(K_RANGE, sil_scores) if s is not None]
axes[1].plot(sil_ks, sil_vals, marker='o', color='orange')
axes[1].set_title('Silhouette Score – Stock Clustering')
axes[1].set_xlabel('k')
axes[1].set_ylabel('Silhouette Score')
plt.tight_layout()
plt.show()

### Fit K-Means & Visualise with PCA

In [ ]:
K_STOCKS = 5  # adjust based on elbow / silhouette above

km_stocks = KMeans(n_clusters=K_STOCKS, random_state=RANDOM_STATE, n_init=20)
ticker_stats.loc[X_tickers.index, 'cluster'] = km_stocks.fit_predict(X_tickers_s)
ticker_stats['cluster'] = ticker_stats['cluster'].astype('Int64')

# PCA for 2-D visualisation
pca = PCA(n_components=2, random_state=RANDOM_STATE)
coords = pca.fit_transform(X_tickers_s)

plot_df = X_tickers.copy()
plot_df['pc1']     = coords[:, 0]
plot_df['pc2']     = coords[:, 1]
plot_df['cluster'] = km_stocks.labels_
plot_df['ticker']  = X_tickers.index

fig, ax = plt.subplots(figsize=(12, 7))
palette = sns.color_palette('tab10', K_STOCKS)
for c in range(K_STOCKS):
    sub = plot_df[plot_df['cluster'] == c]
    ax.scatter(sub['pc1'], sub['pc2'], label=f'Cluster {c}', alpha=0.7, s=40, color=palette[c])

ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} var)')
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} var)')
ax.set_title(f'Stock Clusters in PCA Space  (k={K_STOCKS})')
ax.legend()
plt.tight_layout()
plt.show()

print(f'Cluster sizes:\n{plot_df["cluster"].value_counts().sort_index()}')

### Cluster Profiles

In [ ]:
profile_cols = [
    'mean_return', 'std_return', 'sharpe_approx',
    'mean_hl_range', 'mean_vol_norm',
    'mean_price_to_ma', 'up_rate',
    'log_marketcap', 'revenuegrowth',
]

profiles = (
    ticker_stats.dropna(subset=['cluster'])
    .groupby('cluster')[profile_cols]
    .mean()
    .round(4)
)
print('=== Cluster Mean Profiles ===')
print(profiles.to_string())

# Heatmap (z-scored across clusters)
profiles_z = (profiles - profiles.mean()) / (profiles.std() + 1e-9)
fig, ax = plt.subplots(figsize=(12, 4))
sns.heatmap(
    profiles_z.T, annot=profiles.T.round(4),
    fmt='g', cmap='RdBu_r', center=0,
    ax=ax, cbar_kws={'label': 'z-score'},
)
ax.set_title('Stock Cluster Profiles (z-scored)')
ax.set_xlabel('Cluster')
plt.tight_layout()
plt.show()

### Sector Distribution per Cluster

In [ ]:
sector_cluster = (
    ticker_stats.dropna(subset=['cluster'])
    .groupby(['cluster', 'Sector'])
    .size()
    .unstack(fill_value=0)
)
# Normalise to % within each cluster
sector_pct = sector_cluster.div(sector_cluster.sum(axis=1), axis=0) * 100

sector_pct.plot.bar(stacked=True, figsize=(12, 5), colormap='tab20')
plt.title('Sector Composition per Stock Cluster (%)')
plt.xlabel('Cluster')
plt.ylabel('% of cluster')
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', fontsize=8)
plt.tight_layout()
plt.show()

---
## Part 2 – Market Day Clustering

Aggregate across all tickers each day to capture the macro environment.

In [ ]:
daily_stats = df.groupby('date').agg(
    mean_return    = ('daily_return', 'mean'),
    std_return     = ('daily_return', 'std'),    # cross-sectional dispersion
    breadth        = ('up',           'mean'),   # % tickers that went up
    mean_hl_range  = ('hl_range',     'mean'),
    mean_vol_norm  = ('vol_norm',     'mean'),
    VIX            = ('VIX',          'first'),
    Yield_Spread   = ('Yield_Spread', 'first'),
    sp500          = ('S&P500',       'first'),
).dropna()

# Add S&P 500 daily return
daily_stats['sp500_ret'] = daily_stats['sp500'].pct_change()
daily_stats = daily_stats.dropna()

print(f'Trading days: {len(daily_stats)}')
daily_stats.head()

### Elbow Method & Silhouette Scores

In [ ]:
DAY_FEAT_COLS = [
    'mean_return', 'std_return', 'breadth',
    'mean_hl_range', 'mean_vol_norm',
    'VIX', 'Yield_Spread', 'sp500_ret',
]

X_days = daily_stats[DAY_FEAT_COLS].dropna()
scaler_d = StandardScaler()
X_days_s = scaler_d.fit_transform(X_days)

inertias_d, sil_d = [], []
for k in range(2, 11):
    km = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
    lbl = km.fit_predict(X_days_s)
    inertias_d.append(km.inertia_)
    sil_d.append(silhouette_score(X_days_s, lbl))

fig, axes = plt.subplots(1, 2, figsize=(14, 4))
axes[0].plot(range(2, 11), inertias_d, marker='o')
axes[0].set_title('Elbow Method – Day Clustering')
axes[0].set_xlabel('k')
axes[0].set_ylabel('Inertia')

axes[1].plot(range(2, 11), sil_d, marker='o', color='orange')
axes[1].set_title('Silhouette Score – Day Clustering')
axes[1].set_xlabel('k')
axes[1].set_ylabel('Silhouette Score')
plt.tight_layout()
plt.show()

### Fit K-Means & Timeline

In [ ]:
K_DAYS = 4  # adjust based on elbow / silhouette above

km_days = KMeans(n_clusters=K_DAYS, random_state=RANDOM_STATE, n_init=20)
daily_stats_sub = daily_stats.loc[X_days.index].copy()
daily_stats_sub['cluster'] = km_days.fit_predict(X_days_s)

# Sort clusters by mean return for consistent labelling
cluster_order = (
    daily_stats_sub.groupby('cluster')['mean_return']
    .mean()
    .sort_values()
    .index.tolist()
)
remap = {old: new for new, old in enumerate(cluster_order)}
daily_stats_sub['cluster'] = daily_stats_sub['cluster'].map(remap)

cluster_names = {0: 'Bearish', 1: 'Weak', 2: 'Moderate', 3: 'Bullish'}
daily_stats_sub['regime'] = daily_stats_sub['cluster'].map(cluster_names)

palette = {'Bearish': 'salmon', 'Weak': 'gold', 'Moderate': 'skyblue', 'Bullish': 'seagreen'}

fig, axes = plt.subplots(3, 1, figsize=(16, 10), sharex=True)

# S&P 500 with regime shading
sp500_ts = daily_stats_sub['sp500'].sort_index()
axes[0].plot(sp500_ts.index, sp500_ts.values, lw=0.8, color='black')
axes[0].set_ylabel('S&P 500')
axes[0].set_title('S&P 500 with K-Means Market Day Regimes')
for regime, color in palette.items():
    mask = daily_stats_sub['regime'] == regime
    for date in daily_stats_sub.loc[mask].index:
        axes[0].axvspan(date, date + pd.Timedelta(days=1), alpha=0.15, color=color, lw=0)

# VIX
axes[1].plot(daily_stats_sub.index, daily_stats_sub['VIX'], lw=0.8, color='darkred')
axes[1].set_ylabel('VIX')

# Cluster label timeline
for regime, color in palette.items():
    sub = daily_stats_sub[daily_stats_sub['regime'] == regime]
    axes[2].scatter(sub.index, [regime] * len(sub), c=color, s=8, label=regime)
axes[2].set_ylabel('Regime')
axes[2].legend(loc='upper left', markerscale=3)

plt.tight_layout()
plt.show()

### Day Cluster Profiles

In [ ]:
day_profiles = (
    daily_stats_sub.groupby('regime')[DAY_FEAT_COLS]
    .mean()
    .round(4)
    .loc[['Bearish', 'Weak', 'Moderate', 'Bullish']]
)
print('=== Day Cluster Profiles ===')
print(day_profiles.to_string())

day_profiles_z = (day_profiles - day_profiles.mean()) / (day_profiles.std() + 1e-9)

fig, ax = plt.subplots(figsize=(12, 4))
sns.heatmap(
    day_profiles_z.T,
    annot=day_profiles.T.round(3),
    fmt='g', cmap='RdBu_r', center=0,
    ax=ax, cbar_kws={'label': 'z-score'},
    xticklabels=day_profiles_z.index,
)
ax.set_title('Market Day Cluster Profiles (z-scored)')
plt.tight_layout()
plt.show()

### Compare K-Means Regimes to Existing GMM Regimes

In [ ]:
gmm_daily = df.groupby('date')['Regime_label'].first().dropna()
compare = daily_stats_sub[['regime']].join(gmm_daily, how='inner')

if len(compare) > 0:
    ct = pd.crosstab(compare['Regime_label'], compare['regime'])
    ct_pct = ct.div(ct.sum(axis=1), axis=0) * 100
    
    fig, ax = plt.subplots(figsize=(8, 4))
    sns.heatmap(ct_pct, annot=True, fmt='.0f', cmap='YlOrRd', ax=ax)
    ax.set_title('GMM Regime label vs K-Means Cluster (% of GMM regime days)')
    ax.set_xlabel('K-Means Regime')
    ax.set_ylabel('GMM Regime Label')
    plt.tight_layout()
    plt.show()
else:
    print('No overlapping dates for comparison (GMM data may not cover the full range).')